In [1]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.5f}'.format})

<Figure size 640x480 with 0 Axes>

In [2]:
fF=1
nH=1
C_val = 1/2
L_val = 1

In [43]:
def resonator_C_resonator(Cc, C_val=C_val, L_val=L_val):
    Cc = sq.Capacitor(Cc,   'F')
    C = sq.Capacitor(C_val, 'F')
    L = sq.Inductor (L_val, 'H')
    elements = {(0, 1): [L, C],
                (0, 2): [L, C],
                (1, 2): [Cc], }
    return sq.Circuit(elements)

def resonator_resonator(connected=True):
    C = sq.Capacitor(C_val, 'F')
    L = sq.Inductor (L_val, 'H')
    if connected:
        elements = {(0, 1): [L, C, L, C], }
    else:
        elements = {(0, 1): [L, C],
                    (0, 2): [L, C], }
    return sq.Circuit(elements)

def resonator_resonator_eq(connected=True):
    if connected:
        C = sq.Capacitor(C_val+C_val, 'F')
        L = sq.Inductor ((1/L_val+1/L_val)**-1, 'H')
        elements = {(0, 1): [L, C], }
    else:
        C = sq.Capacitor(C_val, 'F')
        L = sq.Inductor (L_val, 'H')
        elements = {(0, 1): [L, C],
                    (0, 2): [L, C], }
    return sq.Circuit(elements)

def resonator(Cc):
    # CC = sq.Capacitor(Cc, 'fF')
    # C = sq.Capacitor(C_val, 'fF')
    # L = sq.Inductor(L_val,  'nH')
    # elements = {(0, 2): [L],
    #             (0, 1): [C],
    #             (1, 2): [CC]}
    # return sq.Circuit(elements)
    
    if Cc == 0:
        C = sq.Capacitor(C_val, 'F')
        L = sq.Inductor(L_val,  'H')
        elements = {(0, 1): [L, C], }
        return sq.Circuit(elements)
    else:
        # C = sq.Capacitor(1/(1/C_val+1/Cc), 'F')
        C = sq.Capacitor(((C_val+Cc)/(C_val**2+2*C_val*Cc))**-1, 'F')
        L = sq.Inductor (L_val, 'H')
        elements = {(0, 1): [L, C], }
    return sq.Circuit(elements)
    
def shutned_resonator(Cc):
    Cc = sq.Capacitor(Cc, 'F')
    C = sq.Capacitor(C_val, 'F')
    L = sq.Inductor (L_val, 'H')
    elements = {(0, 1): [L, C, Cc], }
    return sq.Circuit(elements)

In [44]:
n_eig = 8
trunc_num = 60
Cc = 1/5

In [71]:
coupled_res = resonator_C_resonator(Cc=Cc)
res = resonator(Cc=Cc)

uncoupled_res = resonator_C_resonator(Cc=0)
res_0 = resonator(Cc=0)

uncoupled_renormalized_res = resonator_C_resonator(Cc=0, C_val=((C_val+Cc)/(C_val**2+2*C_val*Cc))**-1, L_val=L_val)
renormalized_res = resonator(Cc=Cc)

coupled_res.set_trunc_nums([trunc_num, trunc_num])
uncoupled_res.set_trunc_nums([trunc_num, trunc_num])
uncoupled_renormalized_res.set_trunc_nums([trunc_num, trunc_num])
res.set_trunc_nums([trunc_num  ])
res_0.set_trunc_nums([trunc_num])
renormalized_res.set_trunc_nums([trunc_num])

_ = coupled_res.diag(n_eig)
_ = uncoupled_res.diag(n_eig)
_ = uncoupled_renormalized_res.diag(n_eig)
_ = res.diag(n_eig)
_ = res_0.diag(n_eig)
_ = renormalized_res.diag(n_eig)

In [72]:
Q_0 = res_0._charge_op_isolated(0)
Φ_0 = res_0._flux_op_isolated(0)

Q = res._charge_op_isolated(0)
Φ = res._flux_op_isolated(0)

In [73]:
# H_res_0 = 1 / 2 * (1/fF)               * Q_0 ** 2  + 1 / 2 * (1/nH) * Φ_0 ** 2
# H_res   = 1 / 2 * (1/fF) * (1 + 1/Cc)  * Q ** 2    + 1 / 2 * (1/nH) * Φ   ** 2# 
H_res_0 = res_0.hamiltonian()
H_res   = res.hamiltonian()

I = qt.identity(H_res.shape[0])

H_unc_res = qt.tensor(H_res_0, I) + qt.tensor(I, H_res_0)

H_unc_renorm = qt.tensor(H_res, I) + qt.tensor(I, H_res)

H_coup_res = qt.tensor(H_res, I) + qt.tensor(I, H_res) +  (Cc/ ((C_val**2+2*Cc*C_val)*fF)) * qt.tensor(Q, Q)

In [74]:
E = sq_ext.diag(H_coup_res,n_eig=n_eig)[0]
E -= E[0]
E

array([0.00000, 1.05409, 1.41421, 2.10819, 2.46831, 2.82843, 3.16228, 3.52240])

In [75]:
E = coupled_res._efreqs - coupled_res._efreqs[0]
E

array([0.00000, 1.05409, 1.41421, 2.10819, 2.46831, 2.82843, 3.16228, 3.52240])

In [76]:
E_0 = sq_ext.diag(H_unc_renorm,n_eig=n_eig)[0]
E_0 - E_0[0]

array([0.00000, 1.24722, 1.24722, 2.49444, 2.49444, 2.49444, 3.74166, 3.74166])

In [77]:
uncoupled_renormalized_res._efreqs - uncoupled_renormalized_res._efreqs[0]

array([0.00000, 1.24722, 1.24722, 2.49444, 2.49444, 2.49444, 3.74166, 3.74166])

In [87]:
import numpy as np
from scipy.linalg import eigh

# Parameters
hbar = 1   # Planck constant (can be set to 1 for simplicity)
C = ((C_val+Cc)/(C_val**2+2*C_val*Cc))**-1 * fF
L = L_val * nH
omega = 1/np.sqrt(L*C) # Resonant frequency of the LC circuits
g = 2* 0.5*np.sqrt(C/L)*(Cc/((C_val**2+2*Cc*C_val)*fF))    # Coupling strength

# Creation and annihilation operators for a single quantum state
def create(n):
    return np.diag(np.sqrt(np.arange(1, n)), -1)

def annihilate(n):
    return np.diag(np.sqrt(np.arange(1, n)), 1)

# Size of the matrix (for 2 quanta in each mode)
n = 3  # Number of states for each mode (0, 1, 2)

# Create operators for each mode
a1 = np.kron(create(n), np.eye(n))
a1_dagger = np.kron(annihilate(n), np.eye(n))
a2 = np.kron(np.eye(n), create(n))
a2_dagger = np.kron(np.eye(n), annihilate(n))

# Construct the Hamiltonian
H = hbar * omega * (np.dot(a1_dagger, a1) + np.dot(a2_dagger, a2)) + hbar * g * (np.dot(a1_dagger, a2) + np.dot(a1, a2_dagger)+
                                                                                 0*np.dot(a1_dagger, a2_dagger)+0*np.dot(a1, a2))

# Find the eigenvalues (energy levels)
energies = eigh(H, eigvals_only=True)

# Print the energy levels
E = energies#/ 2 / np.pi #/ 1e9
# E -= E[0]
E

array([0.00000, 1.11606, 1.24722, 1.78174, 2.49444, 3.20713, 3.38531, 4.09801, 5.12003])

In [63]:
E

array([0.00000, 1.05409, 1.41421, 2.10819, 2.46831, 2.82843, 3.16228, 3.52240])